In [1]:
import os
import torch
import albumentations
import albumentations.pytorch
import easydict
import pandas as pd
from torchvision import transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
args = easydict.EasyDict({
    "train_data_file_dir": "./data/dirty_mnist_2nd/",
    "train_data_answer_dir": "./data/dirty_mnist_2nd_answer.csv",
})

assert os.path.isdir(args.train_data_file_dir)
assert os.path.isfile(args.train_data_answer_dir)

In [3]:
class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self, image_folder, label_df, transforms):        
        self.image_folder = image_folder   
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)
    
    def __getitem__(self, index):        
        image_fn = self.image_folder +\
            str(self.label_df.iloc[index,0]).zfill(5) + '.png'
                                              
        image = cv2.imread(image_fn, cv2.IMREAD_GRAYSCALE)
        # By default OpenCV uses BGR color space for color images,
        # so we need to convert the image to RGB color space.
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.label_df.iloc[index,1:].values.astype('float')

        if self.transforms:            
            image = self.transforms(image=image)['image'] / 255.0

        return image, label

In [4]:
data = pd.read_csv(args.train_data_answer_dir)

In [5]:
import albumentations.augmentations

In [125]:
transform = albumentations.Compose([
    albumentations.CLAHE(p=1.0),
    # albumentations.RandomRotate90(),
    # albumentations.VerticalFlip(),       
    
    # albumentations.OneOf([
    #     albumentations.GridDistortion(distort_limit=(-0.3, 0.3), border_mode=cv2.BORDER_CONSTANT),
    #     albumentations.ShiftScaleRotate(rotate_limit=15, border_mode=cv2.BORDER_CONSTANT, p=1),        
    #     albumentations.ElasticTransform(alpha_affine=10, border_mode=cv2.BORDER_CONSTANT, p=1),
    # ], p=1),    
    # albumentations.Cutout(num_holes=16, max_h_size=15, max_w_size=15, fill_value=0),
    albumentations.pytorch.ToTensorV2(),
])

In [126]:
dataset = DatasetMNIST(
    image_folder=args.train_data_file_dir,
    label_df=data.iloc[:300],
    transforms=transform
)

In [127]:
num_samples = 5

fig, ax = plt.subplots(1, num_samples, figsize=(25, 5))

for i in range(num_samples):
  ax[i].imshow(transforms.ToPILImage()(dataset[0][0]), 'gray')
  ax[i].axis('off')